In [2]:
import requests

def get_velib_data():
    api_url = 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records?limit=20'

    try:
        response = requests.get(api_url)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            return {'error': f'Failed to fetch data. Status code: {response.status_code}'}

    except Exception as e:
        return {'error': f'An error occurred: {str(e)}'}

if __name__ == '__main__':
    velib_data = get_velib_data()
    
    if 'error' in velib_data:
        print(f"Error: {velib_data['error']}")
    else:
        print("Velib Data:")
        print(velib_data)


Velib Data:
{'total_count': 1471, 'results': [{'stationcode': '12109', 'name': 'Mairie du 12ème', 'is_installed': 'OUI', 'capacity': 30, 'numdocksavailable': 1, 'numbikesavailable': 27, 'mechanical': 26, 'ebike': 1, 'is_renting': 'OUI', 'is_returning': 'OUI', 'duedate': '2023-11-19T23:19:49+00:00', 'coordonnees_geo': {'lon': 2.3875549435616, 'lat': 48.840855311763}, 'nom_arrondissement_communes': 'Paris', 'code_insee_commune': None}, {'stationcode': '14014', 'name': 'Jourdan - Stade Charléty', 'is_installed': 'OUI', 'capacity': 60, 'numdocksavailable': 42, 'numbikesavailable': 17, 'mechanical': 11, 'ebike': 6, 'is_renting': 'OUI', 'is_returning': 'OUI', 'duedate': '2023-11-19T23:16:23+00:00', 'coordonnees_geo': {'lon': 2.3433353751898, 'lat': 48.819428333369}, 'nom_arrondissement_communes': 'Paris', 'code_insee_commune': None}, {'stationcode': '13007', 'name': 'Le Brun - Gobelins', 'is_installed': 'OUI', 'capacity': 48, 'numdocksavailable': 30, 'numbikesavailable': 16, 'mechanical': 13

In [3]:
import requests

# Fonction pour obtenir les données Velib
def get_velib_data():
    api_url = 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records?limit=20'

    try:
        response = requests.get(api_url)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            return {'error': f'Failed to fetch data. Status code: {response.status_code}'}

    except Exception as e:
        return {'error': f'An error occurred: {str(e)}'}

# Récupération des données Velib
velib_data = get_velib_data()

if 'error' in velib_data:
    print(f"Error: {velib_data['error']}")
else:
    # Stockage des données Velib dans une liste de dictionnaires
    velib_stations = []
    for record in velib_data['results']:
        station_data = {
            'stationcode': record['stationcode'],
            'name': record['name'],
            'capacity': record['capacity'],
            'numdocksavailable': record['numdocksavailable'],
            'numbikesavailable': record['numbikesavailable'],
            'mechanical': record['mechanical'],
            'ebike': record['ebike'],
            'is_renting': record['is_renting'],
            'is_returning': record['is_returning'],
            'duedate': record['duedate'],
            'lon': record['coordonnees_geo']['lon'],
            'lat': record['coordonnees_geo']['lat']
        }
        velib_stations.append(station_data)

    # Affichage des données Velib
    for station in velib_stations:
        print(station)

# Vous pouvez maintenant utiliser la liste velib_stations pour effectuer des opérations ou analyses supplémentaires.


{'stationcode': '12109', 'name': 'Mairie du 12ème', 'capacity': 30, 'numdocksavailable': 1, 'numbikesavailable': 27, 'mechanical': 26, 'ebike': 1, 'is_renting': 'OUI', 'is_returning': 'OUI', 'duedate': '2023-11-19T23:19:49+00:00', 'lon': 2.3875549435616, 'lat': 48.840855311763}
{'stationcode': '14014', 'name': 'Jourdan - Stade Charléty', 'capacity': 60, 'numdocksavailable': 42, 'numbikesavailable': 17, 'mechanical': 11, 'ebike': 6, 'is_renting': 'OUI', 'is_returning': 'OUI', 'duedate': '2023-11-19T23:16:23+00:00', 'lon': 2.3433353751898, 'lat': 48.819428333369}
{'stationcode': '13007', 'name': 'Le Brun - Gobelins', 'capacity': 48, 'numdocksavailable': 30, 'numbikesavailable': 16, 'mechanical': 13, 'ebike': 3, 'is_renting': 'OUI', 'is_returning': 'OUI', 'duedate': '2023-11-19T23:17:49+00:00', 'lon': 2.3534681351338, 'lat': 48.835092787824}
{'stationcode': '7002', 'name': 'Vaneau - Sèvres', 'capacity': 35, 'numdocksavailable': 14, 'numbikesavailable': 20, 'mechanical': 13, 'ebike': 7, 'i

In [5]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from flask import Flask, request, jsonify, render_template
import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.distance import geodesic

app = Flask(__name__)

def get_velib_data():
    api_url = 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records?limit=1000'
    
    try:
        response = requests.get(api_url)
        
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            return None
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

def get_closest_velib_stations(user_location, num_stations=5):
    velib_data = get_velib_data()
    
    if velib_data is None:
        return {'error': 'Failed to fetch Velib data'}
    
    user_lat = user_location['lat']
    user_lon = user_location['lon']
    
    stations = velib_data['records']
    
    # Calculate distances to all stations and store them in a list
    distances = []
    for station in stations:
        station_lat = station['coordonnees_geo']['lat']
        station_lon = station['coordonnees_geo']['lon']
        station_location = (station_lat, station_lon)
        distance = geodesic(user_location, station_location).meters
        distances.append((station, distance))
    
    # Sort stations by distance and select the top num_stations
    sorted_stations = sorted(distances, key=lambda x: x[1])
    closest_stations = sorted_stations[:num_stations]
    
    result = []
    for station, distance in closest_stations:
        station_info = {
            'name': station['name'],
            'distance_meters': distance,
            'num_bikes_available': int(station['numbikesavailable']),
            'capacity': int(station['capacity'])
        }
        result.append(station_info)
    
    return result

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/closest_velib_stations', methods=['POST'])
def closest_velib_stations():
    try:
        user_location = request.form.get('location')
        if user_location:
            geolocator = Nominatim(user_agent="geoapiExercises")
            location = geolocator.geocode(user_location)
            if location:
                user_coordinates = {'lat': location.latitude, 'lon': location.longitude}
                closest_stations = get_closest_velib_stations(user_coordinates)
                return jsonify({'closest_stations': closest_stations})
            else:
                return jsonify({'error': 'Location not found'}), 400
        else:
            return jsonify({'error': 'Invalid user location data'}), 400
    except GeocoderTimedOut:
        return jsonify({'error': 'Geocoding service timed out'}), 500

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Nov/2023 01:04:19] "GET / HTTP/1.1" 200 -
